In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.models import Model,Input

from keras import optimizers
import matplotlib.pyplot as plt
from keras.layers import Dense,concatenate
import random

Using TensorFlow backend.
C:\Users\nur\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nur\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nur\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nur\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [2]:
class data_ops(object):
    def __init__(self, path,file_name,sep,use_cols):
        self.path = path #path of the file
        self.file_name = file_name #the data file name
        self.sep = sep # delimiter if any
        self.columns = use_cols  # which columns to use in this dataset

    def read_data(self):
        self.data = pd.read_csv(self.path,sep = self.sep, usecols = self.columns) # read the dataset from the given path
        self.data.columns = self.columns #filter out unwanted columns

    #This method scales the data into 0,1 scale
    def scale_data(self): 
        self.scaler = MinMaxScaler() # minmaxscaler object to scale our data
        target_val = self.data[self.columns[2]].values #our target value is the last column
        target_val = target_val.reshape(-1,1) # Reshape the target value to [n,1] to avoid shape error
        self.scaler.fit(target_val) # fit the target values to scaler object
        self.data[self.columns[2]] = self.scaler.transform(target_val) # replace unscaled target value with the scaled one.

    def create_train_matrix(self): # create training matrix for the network
        grouped_data  = self.data.groupby([self.columns[0],self.columns[1]])[self.columns[2]].sum().unstack().fillna(0)
        self.train_matrix = grouped_data.values 

    def generator_IE(self):
        #print(np.array(self.train_matrix).shape)
        for row in range(len(self.train_matrix)):
            item = (np.array(self.train_matrix[row]))
            print("output " , (item,item))
            yield (item,item)
    
    def get_data(self):
        self.read_data()
        self.scale_data()
        self.create_train_matrix()
       

In [27]:
class Models(object):

    def __init__(self, gen):
        self.generator = gen
        self.item_size = 4
       
    def item_Encoder(self): 
        input = Input(shape=(1,))
        code = Dense(self.item_size*2, activation="relu", name='IE_hidden1')(input)
        code = Dense(int(self.item_size*2), activation="relu",name='IE_hidden2')(code)
        self.item_input = input
        return code

    def item_Decoder(self,code):    
        output = Dense(int(self.item_size*2), activation="relu",name='ID_hidden3')(code)
        output=  Dense(1, activation="relu",name='ID_hidden4')(output)
        return output

    def model_train_IE(self):
       
        self.item_code = self.item_Encoder()
        item_dec = self.item_Decoder(self.item_code)
        self.imodel = Model(inputs=self.item_input, outputs=item_dec)
        
        self.imodel.compile(loss='mse',optimizer='sgd',metrics=["mse"])
        self.imodel.fit(x=gen,epochs=5,steps_per_epoch=1)
    

In [28]:
path = "deneme.csv"
fname = "deneme.csv"
dobject = data_ops(path,fname,";",["STORE_ID","ITEM_CODE","SALES_QUANTITY"])
dobject.get_data()
gen = dobject.generator_IE()

ae = Models(gen)
ae.model_train_IE()

output  (array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.28825623, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.28825623, 0.        ]))
output  (array([0., 0., 0., 0., 0., 1., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0.]))
output  (array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]))
output  (array([0.        , 0.        , 0.        , 0.        , 0.28825623,
       0.        , 0.        , 0.        ]), array([0.        , 0.        , 0.        , 0.        , 0.28825623,
       0.        , 0.        , 0.        ]))
output  (array([0., 0., 0., 0., 0., 1., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0.]))
output  (array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]))
output  (array([0.        , 0.        , 0.        , 0.28825623, 0.        ,
       0.        , 0.        , 0.        ]), array([0.        , 0.        , 0.        , 0.2882

In [29]:
print(ae.imodel.summary())

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
IE_hidden1 (Dense)           (None, 8)                 16        
_________________________________________________________________
IE_hidden2 (Dense)           (None, 8)                 72        
_________________________________________________________________
ID_hidden3 (Dense)           (None, 8)                 72        
_________________________________________________________________
ID_hidden4 (Dense)           (None, 1)                 9         
Total params: 169
Trainable params: 169
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
np.array(next(dobject.generator_IE())).shape

output  (array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.28825623, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.28825623, 0.        ]))


(2, 8)